In [1]:
import pandas as pd

In [2]:
df_calendar = pd.read_csv('Downloads/calendar.csv')
df_listing = pd.read_csv('Downloads/listings.csv')

/Users/wngauv/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (43,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_calendar.columns = ['id', 'date', 'available', 'price']
df_calendar['date'] = pd.to_datetime(df_calendar['date'], format='%Y-%m-%d')

In [4]:
df_calendar = df_calendar.loc[df_calendar['available'] == 'f']

In [5]:
df_calendar.head()

,id,date,available,price
0,2525,2018-04-03,f,NaN
1,2525,2018-04-02,f,NaN
2,2525,2018-04-01,f,NaN
3,2525,2018-03-31,f,NaN
4,2525,2018-03-30,f,NaN


In [6]:
agg_duration = df_calendar.groupby(['id']).size().reset_index()
agg_duration.columns = ['id', 'duration']

In [14]:
df_listing['bed_type'].value_counts()

Real Bed         50678
Pull-out Sofa     4499
Couch              940
Futon              382
Airbed              36
Name: bed_type, dtype: int64

In [15]:
df_final = df_listing.merge(agg_duration, on='id', how='left')

In [17]:
idlisting = df_listing['id'].unique().tolist()

In [21]:
idcalendar = df_calendar['id'].unique().tolist()

In [23]:
len(list(set(idlisting) & set(idcalendar)))

53330

In [26]:
df_final.to_csv('Downloads/offre_durationbyID.csv', sep=";", index=False)

In [27]:
df_loyer = pd.read_csv('Downloads/encadrement_loyers.csv')

In [29]:
df_loyer = df_loyer.loc[df_loyer['ville'] == 'PARIS']

In [31]:
df_loyer.head()

,ville,id_zone,code_grand_quartier,code_iris,nom_quartier,piece,epoque,meuble,annee,ref,min,max
0,PARIS,2,7510101,NaN,St-Germain-l'Auxerrois,1,Avant 1946,0,2015,30.3,21.20,36.40
1,PARIS,2,7510101,NaN,St-Germain-l'Auxerrois,1,Avant 1946,0,2016,30.5,21.35,36.60
2,PARIS,2,7510101,NaN,St-Germain-l'Auxerrois,1,Avant 1946,0,2017,30.6,21.40,36.70
3,PARIS,2,7510101,NaN,St-Germain-l'Auxerrois,1,Avant 1946,1,2015,33.6,23.50,40.30
4,PARIS,2,7510101,NaN,St-Germain-l'Auxerrois,1,Avant 1946,1,2016,34.2,23.94,41.04


In [33]:
df_final.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,duration
0,3508970,https://www.airbnb.com/rooms/3508970,20170404145355,2017-04-06,Cosy Aptmt Bastille - Gare de Lyon,Cosy petit appartement de 60m2 situé aux pieds...,NaN,Cosy petit appartement de 60m2 situé aux pieds...,none,"Marché d'Aligre, Coulée verte, Bastille, Squar...",...,f,NaN,Paris,f,moderate,f,f,1,0.46,83.0
1,13222966,https://www.airbnb.com/rooms/13222966,20170404145355,2017-04-06,Modern & bright Haussmann STUDIO (25m2 +mezzan...,This bright studio is located in an Haussmann ...,NaN,This bright studio is located in an Haussmann ...,none,NaN,...,f,NaN,Paris,f,flexible,f,f,1,0.30,365.0
2,7337128,https://www.airbnb.com/rooms/7337128,20170404145355,2017-04-06,"One flat in Nation, Paris 12th",The apartment is located in Nation (Paris’ eas...,NaN,The apartment is located in Nation (Paris’ eas...,none,NaN,...,f,NaN,Paris,f,strict,f,f,1,0.58,NaN
3,5764597,https://www.airbnb.com/rooms/5764597,20170404145355,2017-04-06,Studio 32 M² avec Balcon à Paris 12,Bonjour !,Je vous propose un studio lumineux de 32m2 sit...,Bonjour ! Je vous propose un studio lumineux d...,none,"De nombreux restaurants sont à proximité, ains...",...,f,NaN,Paris,f,flexible,f,f,1,1.21,363.0
4,7861852,https://www.airbnb.com/rooms/7861852,20170404145355,2017-04-06,Large private bedroom - Paris,Large bedroom (15 m²) in a spacious apartment ...,The bedroom is 15m² large and is equipped with...,Large bedroom (15 m²) in a spacious apartment ...,none,The apartment is located 10 min by metro from ...,...,f,NaN,Paris,f,flexible,f,f,1,0.06,365.0


In [42]:
df_loyer.loc[df_loyer['nom_quartier'] == 'Nation']

,ville,id_zone,code_grand_quartier,code_iris,nom_quartier,piece,epoque,meuble,annee,ref,min,max


In [50]:
df_loyer['nom_quartier'].value_counts()

Batignolles              96
Maison-Blanche           96
Javel 15Art              96
Villette                 96
Roquette                 96
Picpus                   96
Belleville               96
Ternes                   96
Gaillon                  96
Archives                 96
Quinze-Vingts            96
Porte-Saint-Martin       96
Mail                     96
Gros-Caillou             96
Auteuil                  96
Vivienne                 96
Odeon                    96
Europe                   96
Place-Vendôme            96
Bonne-Nouvelle           96
Parc-de-Montsouris       96
Halles                   96
Charonne                 96
Saint-Merri              96
Sainte-Marguerite        96
Epinettes                96
Père-Lachaise            96
Palais-Royal             96
Chaillot                 96
Invalides                96
                         ..
Porte-Dauphine           96
Pont-de-Flandre          96
Saint-Fargeau            96
Saint-Gervais            96
Clignancourt        

In [54]:
df_final['host_neighbourhood'].value_counts()

Montmartre                           3185
République                           2643
Buttes-Chaumont - Belleville         2520
Le Marais                            2200
Batignolles                          1955
Bastille                             1826
XI Arrondissement                    1722
Alésia                               1651
Gare du Nord - Gare de I'Est         1566
Châtelet - Les Halles - Beaubourg    1562
Saint-Germain-des-Prés - Odéon       1532
Place d'Italie - Quartier Chinois    1413
Père Lachaise - Ménilmontant         1377
Passy                                1369
La Chapelle                          1360
Pigalle - Saint-Georges              1357
Nation                               1245
Tour Eiffel - Champ de Mars          1239
Porte de Clignancourt                1092
Panthéon                             1069
Commerce - Dupleix                   1000
La Villette                           972
Porte de Versailles                   965
Champs-Elysées                    

In [34]:
df_final.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

In [4]:
df_quartier = pd.read_csv('Downloads/osm_paris_quartiers.csv', sep=";", usecols=["osm_id", "name",
                                     "addr.postcode", "admin_level",
                                     "alt_name", "boundary",
                                     "ref.INSEE", "short_name", "type",
                                     "wikidata", "wikipedia"])

In [2]:
df_loyer = pd.read_csv('Downloads/encadrement_loyers.csv')

In [4]:
df_final = pd.read_csv('Downloads/offre_durationbyID.csv', sep=";")

/Users/wngauv/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,2,27,33,40,43,53,54,65,67,68,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_loyer.columns

Index(['ville', 'id_zone', 'code_grand_quartier', 'code_iris', 'nom_quartier',
       'piece', 'epoque', 'meuble', 'annee', 'ref', 'min', 'max'],
      dtype='object')

In [7]:
df_loyer = df_loyer.loc[df_loyer['annee'] >= 2017]

In [8]:
df_loyer.loc[df_loyer['nom_quartier'].str.contains('Sorbonne')]

,ville,id_zone,code_grand_quartier,code_iris,nom_quartier,piece,epoque,meuble,annee,ref,min,max
1826,PARIS,4,7510520,NaN,Sorbonne,1,Avant 1946,0,2017,31.1,21.8,37.3
1829,PARIS,4,7510520,NaN,Sorbonne,1,Avant 1946,1,2017,34.8,24.4,41.8
1832,PARIS,4,7510520,NaN,Sorbonne,1,1946-1970,0,2017,28.1,19.7,33.7
1835,PARIS,4,7510520,NaN,Sorbonne,1,1946-1970,1,2017,31.5,22.1,37.8
1838,PARIS,4,7510520,NaN,Sorbonne,1,1971-1990,0,2017,28.1,19.7,33.7
1841,PARIS,4,7510520,NaN,Sorbonne,1,1971-1990,1,2017,31.5,22.1,37.8
1844,PARIS,4,7510520,NaN,Sorbonne,1,Apres 1990,0,2017,28.4,19.9,34.1
1847,PARIS,4,7510520,NaN,Sorbonne,1,Apres 1990,1,2017,31.8,22.3,38.2
1850,PARIS,4,7510520,NaN,Sorbonne,2,Avant 1946,0,2017,26.1,18.3,31.3
1853,PARIS,4,7510520,NaN,Sorbonne,2,Avant 1946,1,2017,29.2,20.4,35.0


In [5]:
df_quartier

,osm_id,name,addr.postcode,admin_level,alt_name,boundary,ref.INSEE,short_name,type,wikidata,wikipedia
0,2171716,Quartier du Bel-Air,75012,10,NaN,administrative,7511245,Bel-Air,boundary,Q3413267,fr:Quartier du Bel-Air
1,2171718,Quartier de Bercy,75012,10,NaN,administrative,7511247,Bercy,boundary,Q773318,fr:Quartier de Bercy
2,2171720,Quartier de Picpus,75012,10,NaN,administrative,7511246,Picpus,boundary,Q3413171,fr:Quartier de Picpus
3,2172736,Quartier de l'Arsenal,75004,10,NaN,administrative,7510415,Arsenal,boundary,Q2426361,fr:Quartier de l'Arsenal
4,2172737,Quartier Les Halles,75001,10,NaN,administrative,7510102,Les Halles,boundary,Q387285,fr:Quartier des Halles
5,2172738,Quartier Les Îles,75004,10,Quartier Notre-Dame,administrative,7510416,Îles,boundary,Q3413095,fr:Quartier Notre-Dame (Paris)
6,2172739,Quartier du Palais Royal,75001,10,NaN,administrative,7510103,Palais Royal,boundary,Q1763626,fr:Quartier du Palais-Royal
7,2172740,Quartier Saint-Germain-l'Auxerrois,75001,10,NaN,administrative,7510101,St-Germain-l'Auxerrois,boundary,Q2409554,fr:Quartier Saint-Germain-l'Auxerrois
8,2172741,Quartier Saint-Gervais,75004,10,NaN,administrative,7510414,St-Gervais,boundary,Q3413113,fr:Quartier Saint-Gervais
9,2172742,Quartier Saint-Merri,75004,10,NaN,administrative,7510413,St-Merri,boundary,Q3413122,fr:Quartier Saint-Merri


In [28]:
df_quartier.head()

,osm_id,name,addr.postcode,admin_level,alt_name,boundary,ref.INSEE,short_name,type,wikidata,wikipedia
0,2171716,Quartier du Bel-Air,75012,10.0,NaN,administrative,7511245.0,Bel-Air,boundary,Q3413267,fr:Quartier du Bel-Air
1,"657610.792735506, 657622.756834254, 657630.831...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"658118.733292198, 658114.641387961, 658112.397...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"657372.19057389, 657358.451260677, 657342.9821...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"656616.091356478, 656600.48747457, 656584.4891...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
